In [1]:
import pandas as pd
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import json
import demjson
import re
from selenium import webdriver

In [2]:
main_url = input('Please Insert Search URL :')
response = requests.get(main_url)
# Parse HTML and save to BeautifulSoup object¶
soup = BeautifulSoup(response.content, 'html.parser')


Please Insert Search URL :https://www.hipflat.co.th/en/search/sale/condo_y/TH.BM_r1/TH.BM.WA_r2/any_p/any_b/any_a/any_w/any_i/100.58276002199337,13.734960866224068_c/14_z/list_v


In [3]:
n_pages = input('How Many Pages Do you want to scrap? :')
n_pages = int(n_pages)

How Many Pages Do you want to scrap? :10


In [4]:
driver = webdriver.Chrome(r'C:\Users\User\Desktop\Py Scrab\chromedriver.exe')
driver.get(main_url)
path = '//div[@class = "pagination-btn__right"]//a[@data-role = "next"]'
urls = []

for k in range(n_pages) :
    geturl = soup.find_all('div',class_='listing-title')
    for i in geturl :
        for m in i.find_all('h1') :
            geturl = str(m)
            urls.append('https://www.hipflat.co.th'+geturl[14:49])
            
    time.sleep(5) #making time treshold to avoid of 'StaleElementReferenceException'
    driver.find_elements_by_xpath(path)[0].click()
    print('Scraping Url Page ....' + str(k))


Scraping Url Page ....0
Scraping Url Page ....1
Scraping Url Page ....2
Scraping Url Page ....3
Scraping Url Page ....4
Scraping Url Page ....5
Scraping Url Page ....6
Scraping Url Page ....7
Scraping Url Page ....8
Scraping Url Page ....9


In [5]:
def scrab_hipflat(url1) :
    response = requests.get(url1)
    # Parse HTML and save to BeautifulSoup object¶
    soup = BeautifulSoup(response.content, 'html.parser')
    soup.findAll('a')
    time.sleep(1)
    collect = [] #last DataFrame
    real_collect = []


    #collect name of projects
    name = soup.find_all('div',class_='project-header-name')
    collect_name = []
    for i in name :
        i = i.text
        i = str(i)
        i = i.strip()
        collect_name.append(i)
    try :
        condo_name = collect_name[0]
    except :
        condo_name = 'NaN'
    else :
        condo_name = collect_name[0]
    
    
    
    #collect get full descriptions
    description = ''
    try :
        des_clt = soup.find('article',class_='property-description')
        for i in des_clt.find_all('p') :
            description = description + i.text
    
    except :
        description = 'NaN'
    
    else :
        des_clt = soup.find('article',class_='property-description')
        for i in des_clt.find_all('p') :
            description = description + i.text

        
        
    
    #collect price
    price = soup.find_all('span',class_='money')
    inter_price = [] #pre-collect-price
    for i in price :
        i = i.text
        i = str(i)
        i = i.strip()
        i = i[1:]
        inter_price.append(i)
    collect_price = inter_price[0]
    collect_price = collect_price.strip()
    collect_price = collect_price.replace(',', '')

    
    
    #collect details
    details = [] #collect details
    for tag in soup.find_all('ul',{'class' : 'listing-essentials__features listing-essentials__features--three-items'}) :
        for span_tag in tag.find_all('li',{'class' :'listing-essentials__feature'}) :
            k = span_tag.find('span',{'class':'number'})
            k = k.text
            details.append(k)
    try :
        Bedroom = details[0]
        Bathroom = details[1]
        Areas = details[2]
    except :
        Bedroom = 0
        Bathroom = 0
        Areas = 0
    else :
        Bedroom = details[0]
        Bathroom = details[1]
        Areas = details[2]
    
    
    #collect facilities
    try :
        amenities = soup.find('ol',class_ = 'amenities')
        a = amenities.find_all('li')
        collect_facilities = dict()
        for i in a :
                for j in i.find_all('div') :
                    if 'amenities__icon--yes' in j['class'] :
                        collect_facilities[i.text.strip()] = True
                    elif 'amenities__icon--no' in j['class'] :
                        collect_facilities[i.text.strip()] = False
    except :
        collect_facilities = dict()
        collect_facilities['Elevator'] = 'NaN',
        collect_facilities['Parking'] = 'NaN',
        collect_facilities['24-hours Security']= 'NaN',
        collect_facilities['CCTV'] = 'NaN',
        collect_facilities['Swimming Pool'] = 'NaN',
        collect_facilities['Sauna'] = 'NaN',
        collect_facilities['Gym'] = 'NaN',
        collect_facilities['Garden / BBQ'] = 'NaN',
        collect_facilities['''Playground / Kid's Area'''] = 'NaN',
        collect_facilities['Shop on Premise'] = 'NaN',
        collect_facilities['Restaurant on Premise'] = 'NaN',
        collect_facilities['Wi-Fi'] = 'NaN'
    
    else :
        amenities = soup.find('ol',class_ = 'amenities')
        a = amenities.find_all('li')
        collect_facilities = dict()
        for i in a :
                for j in i.find_all('div') :
                    if 'amenities__icon--yes' in j['class'] :
                        collect_facilities[i.text.strip()] = True
                    elif 'amenities__icon--no' in j['class'] :
                        collect_facilities[i.text.strip()] = False


                    
    #format json form
    def clt_json(soup,num) :
        a = soup.find_all('script')[num]
        for i in a :
            i = str(i)
            i = i.strip()
        json1 = i[102:-213] #remove js statement
        json1 = json1.replace('\n','') #remove symbol
        json1 = json1.replace('''\'''','')
        json1 = json1.replace('\\','')
        json1 = json1.replace(' ','')
        json1 = json1.strip()
        json1 = re.sub('projectId','''"projectId"''',json1,1)
        json1 = re.sub('lat','''"lat"''',json1,1)
        json1 = re.sub('lng','''"lng"''',json1,1)
        json1 = re.sub('destinations','''"destinations"''',json1)
        json1 = re.sub(':',':"',json1,1)
        json1 = re.sub(',','",',json1,1)
        rec_json = json.loads(json1) #recieve json from jsonload

        #collect location of condo
        condo_lat = rec_json['lat']
        condo_lng = rec_json['lng']


        #collect location of transportation
        trans_name = []
        trans_dist = []

        #collect location of resturant
        res_name = []
        res_dist = []

        #collect location of hospital
        hos_name = []
        hos_dist = []

        #collect location of school 
        sch_name = []
        sch_dist = []

        #collect location of bank 
        bank_name = []
        bank_dist = []

        #collect destinations in var
        for i in range(len(rec_json['destinations'])) :
            if rec_json['destinations'][i]['category_class_name'] == 'transportation' :
                trans_name.append(rec_json['destinations'][i]['name'])
                trans_dist.append(rec_json['destinations'][i]['distance_human'])
            elif rec_json['destinations'][i]['category_class_name'] == 'restaurants' :
                res_name.append(rec_json['destinations'][i]['name'])
                res_dist.append(rec_json['destinations'][i]['distance_human'])
            elif rec_json['destinations'][i]['category_class_name'] == 'hospitals' :
                hos_name.append(rec_json['destinations'][i]['name'])
                hos_dist.append(rec_json['destinations'][i]['distance_human'])
            elif rec_json['destinations'][i]['category_class_name'] == 'schools' :
                sch_name.append(rec_json['destinations'][i]['name'])
                sch_dist.append(rec_json['destinations'][i]['distance_human'])
            elif rec_json['destinations'][i]['category_class_name'] == 'banks' :
                bank_name.append(rec_json['destinations'][i]['name'])
                bank_dist.append(rec_json['destinations'][i]['distance_human'])

        #export to series
        row = [description,url1,condo_name,collect_price,Bedroom,Bathroom,Areas,condo_lat,condo_lng,trans_name[0],trans_dist[0],res_name[0],res_dist[0],hos_name[0],hos_dist[0],sch_name[0],sch_dist[0],bank_name[0],bank_dist[0],
                    collect_facilities['Elevator'],collect_facilities['Parking'],collect_facilities['24-hours Security'],collect_facilities['CCTV'],collect_facilities['Swimming Pool'],collect_facilities['Sauna'],collect_facilities['Gym'],
                    collect_facilities['Garden / BBQ'],collect_facilities['''Playground / Kid's Area'''],collect_facilities['Shop on Premise'],collect_facilities['Restaurant on Premise'],collect_facilities['Wi-Fi']]
        return row
    #Try to avoid app crash
    try :
        clt_json(soup,19)
    except :
        try :
            clt_json(soup,20)
        except :
            row = [description,url1,condo_name,collect_price,Bedroom,Bathroom,Areas,0,0,0,0,0,0,0,0,0,0,0,0,collect_facilities['Elevator'],collect_facilities['Parking'],collect_facilities['24-hours Security'],collect_facilities['CCTV'],collect_facilities['Swimming Pool'],collect_facilities['Sauna'],collect_facilities['Gym'],
                    collect_facilities['Garden / BBQ'],collect_facilities['''Playground / Kid's Area'''],collect_facilities['Shop on Premise'],collect_facilities['Restaurant on Premise'],collect_facilities['Wi-Fi']]
        else :
            row = clt_json(soup,20)
    else :
        row = clt_json(soup,19)
    
    return row




In [ ]:
dataset = []
i = 0
for url in urls :
    row1 = scrab_hipflat(url)
    print(url)
    dataset.append(row1)
    i +=1
    print('Scrapped ' + str(i) 
         + ' data')

https://www.hipflat.co.th/en/listings/bangkok-condo-vrjckpwo
Scrapped 1 data
https://www.hipflat.co.th/en/listings/bangkok-condo-mxuqjbaw
Scrapped 2 data
https://www.hipflat.co.th/en/listings/bangkok-condo-qfmpfloo
Scrapped 3 data
https://www.hipflat.co.th/en/listings/bangkok-condo-rdhnhwfj
Scrapped 4 data
https://www.hipflat.co.th/en/listings/bangkok-condo-igqpbyxk
Scrapped 5 data
https://www.hipflat.co.th/en/listings/bangkok-condo-lxlaimuz
Scrapped 6 data
https://www.hipflat.co.th/en/listings/bangkok-condo-naasowzz
Scrapped 7 data
https://www.hipflat.co.th/en/listings/bangkok-condo-xxmjxfgs
Scrapped 8 data
https://www.hipflat.co.th/en/listings/bangkok-condo-tuzjdmcp
Scrapped 9 data
https://www.hipflat.co.th/en/listings/bangkok-condo-ujxavqif
Scrapped 10 data
https://www.hipflat.co.th/en/listings/bangkok-condo-kolpnywv
Scrapped 11 data
https://www.hipflat.co.th/en/listings/bangkok-condo-hienihka
Scrapped 12 data
https://www.hipflat.co.th/en/listings/bangkok-condo-rcuyebmw
Scrapped 13 

https://www.hipflat.co.th/en/listings/bangkok-condo-fxvezzun
Scrapped 107 data
https://www.hipflat.co.th/en/listings/bangkok-condo-ktoeyrop
Scrapped 108 data
https://www.hipflat.co.th/en/listings/bangkok-condo-akohfzhv
Scrapped 109 data
https://www.hipflat.co.th/en/listings/bangkok-condo-rxdbserl
Scrapped 110 data
https://www.hipflat.co.th/en/listings/bangkok-condo-nlqjmpkz
Scrapped 111 data
https://www.hipflat.co.th/en/listings/bangkok-condo-wdrillbb
Scrapped 112 data
https://www.hipflat.co.th/en/listings/bangkok-condo-zpfnscfh
Scrapped 113 data
https://www.hipflat.co.th/en/listings/bangkok-condo-nwqeivgm
Scrapped 114 data
https://www.hipflat.co.th/en/listings/bangkok-condo-lldaqnse
Scrapped 115 data
https://www.hipflat.co.th/en/listings/bangkok-condo-mrozzjex
Scrapped 116 data
https://www.hipflat.co.th/en/listings/bangkok-condo-dtuvbwfl
Scrapped 117 data
https://www.hipflat.co.th/en/listings/bangkok-condo-bhyktddm
Scrapped 118 data
https://www.hipflat.co.th/en/listings/bangkok-condo-

In [ ]:
#convert to csv
df = pd.DataFrame(dataset,columns = ['description','url','condo_name','price','Bedroom','Bathroom','Areas','condo_lat','condo_lng','trans_name','trans_dist','res_name','res_dist','hos_name','hos_dist','sch_name','sch_dist','bank_name','bank_dist','Elevator','Parking','24-hr Security','CCTV','Swimming_Pool','Sauna','Gym','Garden','Platground','Shop','Restaurant','Wi-Fi'])

In [ ]:
print(df['description'])

In [ ]:
df.to_csv('hipflat_scrap.csv', sep = ',')